# Run the final state algorithm on a ROOT HSDATA file

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.
The string arguements set which particles you would like to use their pre-determined PID for and which you would like to be inclusive (i.e. any number of thse particles).
**Example**, 
1. for tagged photon experiment I might want fs("Beam","Beam")
This means use predetermined photon ID (i.e. I know it came from the tagger) and I will analyse events with any number of tagger photons.
2. fs("e-","proton:pi+:pi-")  i.e. use predetemined electron PID and just determine hadron species based on charge and try all combinations in events with any number of proton,pi+,pi-, but with the number of electrons defined for the topology
3. fs("NONE","ALL") => do not use any PID info other than charge, analyse events with any number of particles (this can be limited by SetMaxParticles)

In [ ]:
gROOT->ProcessLine(".x LoadXXX.C+"); //Load the classes

In [ ]:
 XXX fs("NONE","ALL"); //string arguments => PID, INCLUSIVE
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [ ]:
  //strings = treename and filename (give full path)
  auto output = new FiledTree("FinalTree","OutPUT.root");
  fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

We can apply some broad delta time cuts to remove any complete junk events

In [ ]:
ParticleCutsManager cutsman;
auto otherCut=make_shared<DeltaTimeCut>(20); //20ns cut for all other particles
cutsman.SetDefaultCut(otherCut); //assign to manager
cutsman.ConfigureCuts(&fs);    //propogate the cuts to topologies in fs

**Particle Data** here we are going to make seperate trees for each topology and only keep the detected particle information in each one. These trees are then used to train the classifiers

In [ ]:
auto treeman=new ParticleDataManager("/path/to/save/particlesPD/");
//treeman->SetParticleVars("Beam",{"P","Time"}); //For beam only interesetef in energy and time
treeman->ConfigureTreeParticles(&fs); //propogate through topologies
//treeman.SetN(50000); //Number of events required for each topo. Number of train+test events must be less than this
fs.RegisterPostWorkAction(treeman); //register post-work i.e. after kinematics is called and all calculations

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager dereived classes. 
For this TMVA training we need to load simulated events to act as signal, we may use data or simulated background as backgound. The AddParticle() function in XXX.C must be configured to read the generated truth values properly e.g.

      AddParticle("Beam",&fBeam,kTRUE,-1);
      AddParticle("Proton",&fProton,kTRUE,0);
      AddParticle("Kp",&fKp,kTRUE,1);
      AddParticle("Km",&fKm,kTRUE,2);

Where -1 => Beam not in MCevent; 0,1,2 => Proton, Kp, Km are the first 3 particles in the generated event list.

In [ ]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/indir/expData_*root");  //Add background events
  chain.Add("/simdir/simData_*root")  //Add signal  events
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [ ]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;
fs.ProcessData(); //No number give, analyse all events in chain

Draw some variables from your tree. e.g. You will need to change to match your own particles

In [ ]:
TCanvas c3;
c3.Divide(2,2);
c3.cd(1);
treeman->GetParticleData(0)->GetTree()->Draw("BeamP:BeamTime>>h1(100,-10,10,100,0.4)","","col1");
c3.cd(2);
treeman->GetParticleData(0)->GetTree()->Draw("ProtonP:ProtonTime>>h2(100,-10,10,100,0,4)","","col1");
c3.cd(3);
treeman->GetParticleData(0)->GetTree()->Draw("KpP:KpTime>>h3(100,-10,10,100,0,4)","","col1");
c3.cd(4);
treeman->GetParticleData(0)->GetTree()->Draw("KmP:KmTime>>h4(100,-10,10,100,0,4)","","col1");


Now train TMVA classifiers

In [ ]:
gROOT->ProcessLine(".x $HSCODE/hsmva/LoadHSMVA.C+");

In [ ]:
 for(UInt_t ip=0;ip<treeman->NData();ip++ ){ //Loop over topologies
   auto pd=treeman->GetParticleData(ip);

   auto trainer=  new TrainSignalID(Form("Topo%d",ip),"!V:Silent:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification");
     
   trainer->SetOutDir(treeman->OutDir());
   trainer->IgnoreBranches("Signal:Background"); //ignore these branches for training
   trainer->AddSignalTree(pd->GetTree(),"Signal"); //Use "Signal" branch to filter Signal events
   trainer->AddBackgroundTree(pd->GetTree(),"Background"); //Use "Background" branch for bg events
   trainer->SetNTrainTest(50000,50000); //set number of training and testing events, there must be sufficient number in the trees for each topo
   trainer->PrepareTrees(); 
   //trainer->BookMethod(Meths.BDTA); //choose some classifiers
   trainer->BookMethod(Meths.MLP); //choose some classifiers
   trainer->DoTraining();
   trainer->DrawROCCurve();
   //trainer->DrawResponses();
   delete trainer;
  }

And remember to save the tree!

In [ ]:
delete output;